In [13]:
import os
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
from pandas.plotting import register_matplotlib_converters

import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import xarray as xr
import atlite
from shapely.geometry import Point
from shapely.ops import unary_union
import numpy as np
import rasterio
import shapely

from dask.distributed import Client, LocalCluster

nprocesses = 10
cluster = LocalCluster(n_workers=nprocesses, threads_per_worker=2)
client = Client(cluster, asynchronous=True)

resource={}
resource["dask_kwargs"] = {"scheduler": client}

import PySAM.Pvwattsv8 as pv

/home/pklein/anaconda3/envs/pypsa-za2/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37219 instead
  warnings.warn(


In [ ]:
rsa_supply = gpd.read_file('../data/bundle/rsa_supply_regions.gpkg', layer='1-supply')

In [14]:
cutout = atlite.Cutout(
    path="../cutouts/RSA-2017_20-era5.nc",
    chunks={"time": 100},
)
cutout.data = cutout.data.sel(time=slice('2019-01-01', '2019-12-31'))

# pv_sat = cutout.pv(    
#     panel="CSi",
#     shapes=cutout.grid,
#     tracking="horizontal",
#     per_unit=True, 
#     orientation={"slope": 0.0, "azimuth": 0},
#     **resource,
# )

# pv_ft = cutout.pv(    
#     panel="CSi",
#     shapes=cutout.grid,
#     tracking=None,
#     per_unit=True, 
#     orientation="latitude_optimal", # or {"slope": 0.0, "azimuth": 0},
#     **resource,
# )


In [ ]:
pv_ft.sel(dim_0=100).plot(color='r')
pv_sat.sel(dim_0=100).plot()


In [22]:
cutout.data

<xarray.Dataset>
Dimensions:           (x: 63, y: 47, time: 8760)
Coordinates:
  * x                 (x) float64 16.2 16.5 16.8 17.1 ... 33.9 34.2 34.5 34.8
  * y                 (y) float64 -34.8 -34.5 -34.2 -33.9 ... -21.6 -21.3 -21.0
  * time              (time) datetime64[ns] 2019-01-01 ... 2019-12-31T23:00:00
    lon               (x) float64 dask.array<chunksize=(63,), meta=np.ndarray>
    lat               (y) float64 dask.array<chunksize=(47,), meta=np.ndarray>
Data variables: (12/14)
    height            (y, x) float32 dask.array<chunksize=(47, 63), meta=np.ndarray>
    wnd100m           (time, y, x) float32 dask.array<chunksize=(80, 47, 63), meta=np.ndarray>
    wnd_azimuth       (time, y, x) float32 dask.array<chunksize=(80, 47, 63), meta=np.ndarray>
    roughness         (time, y, x) float32 dask.array<chunksize=(80, 47, 63), meta=np.ndarray>
    influx_toa        (time, y, x) float32 dask.array<chunksize=(80, 47, 63), meta=np.ndarray>
    influx_direct     (time, y, x) float32 dask.array<chunksize=(80, 47, 63), meta=np.ndarray>
    ...                ...
    solar_altitude    (time, y, x) float64 dask.array<chunksize=(80, 47, 63), meta=np.ndarray>
    solar_azimuth     (time, y, x) float64 dask.array<chunksize=(80, 47, 63), meta=np.ndarray>
    temperature       (time, y, x) float32 dask.array<chunksize=(80, 47, 63), meta=np.ndarray>
    soil temperature  (time, y, x) float32 dask.array<chunksize=(80, 47, 63), meta=np.ndarray>
    runoff            (time, y, x) float32 dask.array<chunksize=(80, 47, 63), meta=np.ndarray>
    cf                int64 1
Attributes:
    module:             era5
    prepared_features:  ['height', 'influx', 'wind', 'temperature', 'runoff']
    chunksize_time:     100
    dx:                 0.3
    dy:                 0.3
    Conventions:        CF-1.6
    history:            2023-09-21 06:14:21 GMT by grib_to_netcdf-2.25.1: /op...

In [29]:
import os
import PySAM.ResourceTools as tools  # MOVE BACK TO FILES FOLDER
import PySAM.Windpower as wp
import PySAM.Singleowner as so
import PySAM.Pvwattsv8 as pv

# replace with key and email address from https://developer.nrel.gov/signup/
sam_api_key = '6VaYhZeU4bC1yNMhQ9fzGHjaVjucP6iSQu5MTrhL'
sam_email = 'peter.klein@meridianeconomics.co.za'

    lon = -105.22
    lat = 39.75

In [33]:
cutout.grid

,x,y,geometry
0,16.2,-34.8,"POLYGON ((16.35000 -34.95000, 16.35000 -34.650..."
1,16.5,-34.8,"POLYGON ((16.65000 -34.95000, 16.65000 -34.650..."
2,16.8,-34.8,"POLYGON ((16.95000 -34.95000, 16.95000 -34.650..."
3,17.1,-34.8,"POLYGON ((17.25000 -34.95000, 17.25000 -34.650..."
4,17.4,-34.8,"POLYGON ((17.55000 -34.95000, 17.55000 -34.650..."
...,...,...,...
2956,33.6,-21.0,"POLYGON ((33.75000 -21.15000, 33.75000 -20.850..."
2957,33.9,-21.0,"POLYGON ((34.05000 -21.15000, 34.05000 -20.850..."
2958,34.2,-21.0,"POLYGON ((34.35000 -21.15000, 34.35000 -20.850..."
2959,34.5,-21.0,"POLYGON ((34.65000 -21.15000, 34.65000 -20.850..."


In [35]:
# --- Initialize Solar Resource Fetcher with minimum parameters ---
# See function documentation for full parameter list
for cell in cutout.grid.index[0:5]:
    lon = cutout.grid.loc[cell,'x']
    lat = cutout.grid.loc[cell,'y']

    nsrdbfetcher = tools.FetchResourceFiles(
                    tech='solar',
                    nrel_api_key=sam_api_key,
                    nrel_api_email=sam_email)

    # --- List of (lon, lat) tuples or Shapely points ---
    lon_lats = [(lon, lat)]
    nsrdbfetcher.fetch(lon_lats)

    # --- Get resource data file path ---
    nsrdb_path_dict = nsrdbfetcher.resource_file_paths_dict
    nsrdb_fp = nsrdb_path_dict[lon_lats[0]]


Starting data download for solar using 1 thread workers.
Getting list of available NSRDB files for -34.8, 16.2.
List of available data saved to /home/pklein/Github/pypsa-za2/notebooks/data/PySAM Downloaded Weather Files/nsrdb_data_query_response_-34.8_16.2.json.
No URLS available for -34.8, 16.2.

Starting data download for solar using 1 thread workers.
Getting list of available NSRDB files for -34.8, 16.5.
List of available data saved to /home/pklein/Github/pypsa-za2/notebooks/data/PySAM Downloaded Weather Files/nsrdb_data_query_response_-34.8_16.5.json.
No URLS available for -34.8, 16.5.

Starting data download for solar using 1 thread workers.
Getting list of available NSRDB files for -34.8, 16.8.
List of available data saved to /home/pklein/Github/pypsa-za2/notebooks/data/PySAM Downloaded Weather Files/nsrdb_data_query_response_-34.8_16.8.json.
No URLS available for -34.8, 16.8.

Starting data download for solar using 1 thread workers.
Getting list of available NSRDB files for -34

In [15]:
from dask import delayed, compute

# Assuming your function extract_solar and cutout object are already defined

# Create delayed objects for each function call
tasks = [delayed(extract_solar)(cutout, cell) for cell in cutout.grid.index[0:100]]

# Compute the results in parallel
results = compute(*tasks)

print(results)


82
7
89
91
2540

27
668

76
86
80
70
39
57
85
35
16
79
66
98
74
73
93
94
33
59
77
49
888

62
53
67
45
26
96
0
54
56
31
51
1
37
30
48
17
58
3
71
2920
5522
63

10
5
41

42
23
72
97
44
38
19
92
99
132

46
18
34
14
43
87
4
3261

83
12
69
11
64
28
90
9
36
95
65
52
75
21
24
15
50
78
60
47
84
81
(12.943373363980962, 13.018790025581028, 13.07769182400611, 13.13299877223685, 13.162144337674112, 13.169237395120392, 13.155972403693807, 13.141044322267803, 13.17664941779718, 13.221529755411588, 13.107752967225485, 12.794308323558608, 12.400382850204991, 12.35219568044266, 12.812726964504616, 13.189540335916611, 13.450242528152945, 13.474852926950941, 13.461985833778865, 13.441772067239253, 13.40378705818524, 13.361486180053921, 13.31006186797627, 13.255714310187619, 13.213055073449352, 13.164263204777283, 13.100907311070522, 12.999554229167437, 12.879777642380176, 12.755680852986279, 12.646005177036317, 12.537577610024606, 12.431709261941101, 12.34369959265875, 12.263185634647842, 12.2305546280838

In [16]:
results

(12.943373363980962,
 13.018790025581028,
 13.07769182400611,
 13.13299877223685,
 13.162144337674112,
 13.169237395120392,
 13.155972403693807,
 13.141044322267803,
 13.17664941779718,
 13.221529755411588,
 13.107752967225485,
 12.794308323558608,
 12.400382850204991,
 12.35219568044266,
 12.812726964504616,
 13.189540335916611,
 13.450242528152945,
 13.474852926950941,
 13.461985833778865,
 13.441772067239253,
 13.40378705818524,
 13.361486180053921,
 13.31006186797627,
 13.255714310187619,
 13.213055073449352,
 13.164263204777283,
 13.100907311070522,
 12.999554229167437,
 12.879777642380176,
 12.755680852986279,
 12.646005177036317,
 12.537577610024606,
 12.431709261941101,
 12.34369959265875,
 12.263185634647842,
 12.230554628083821,
 12.213228089506757,
 12.223189411901869,
 12.243116108676363,
 12.263248257994128,
 12.2937444210793,
 12.332096856155822,
 12.35021996218172,
 12.34672703152582,
 12.315063432396029,
 12.280329687015271,
 12.254222923241759,
 12.237437687669205,
 12

In [9]:
def extract_solar(cutout, cell):
    pv_inputs ={
		"system_capacity" : 1*1.2,
		"module_type" : 1,
		"dc_ac_ratio" : 1.2,
		"bifaciality" : 0,
		"array_type" : 0,
		"azimuth" : 0,
		"gcr" : 0.4,
		"soiling" : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
		"losses" : 11.42,
		"en_snowloss" : 0,
		"inv_eff" : 96,
		"batt_simple_enable" : 0,
        'tilt': 30,
	}
    
    lat = cutout.grid.loc[cell,'y']
    long = cutout.grid.loc[cell,'x']
    dn = cutout.data.sel(x=long,y=lat)['influx_direct'].values.tolist()
    dh = cutout.data.sel(x=long,y=lat)['influx_diffuse'].values.tolist()
    tdry = (cutout.data.sel(x=long,y=lat)['temperature'].values-273.15).tolist()
    elev = (cutout.data.sel(x=long,y=lat)['height'].values).tolist()
    
    model = pv.new()#default('PVWattsSingleOwner')
    for k, v in pv_inputs.items():
        if k != 'number_inputs':
            setattr(model.SystemDesign, k, v)
    model.AdjustmentFactors.constant = 0

    wspd = [0]*8760
    dates = pd.date_range('2019-01-01 00:00', '2019-12-31 23:00', freq='H')
    year = list(dates.year)
    month = list(dates.month)
    day = list(dates.day)
    hour = list(dates.hour)
    minute = list(dates.minute)

    model.SolarResource.assign(
        {
            'solar_resource_data':{
                'dn':dn,
                'df':dh,
                'tdry':tdry,
                'wspd':wspd,
                'lat':lat,
                'lon':long,
                'tz':2,
                'elev':100,
                'year':year,
                'month':month,
                'day':day,
                'hour':hour,
                'minute':minute,
                'albedo':0.2,
                "use_wf_albedo":0,
                }
        }
    )
    model.execute()
    print(cell)
    return model.Outputs.capacity_factor